Import Python modules...

In [ ]:
from __future__ import print_function

import os
import sys
import time
import re
from io import StringIO
import pandas as pd

import requests

print(time.asctime())

**The URL PATH**

The MW REST URL consists of three main parts, separated by forward slashes, after the common prefix specifying the invariant base URL (https://www.metabolomicsworkbench.org/rest/):

https://www.metabolomicsworkbench.org/rest/context/input_specification/output_specification
    
Part 1: The context determines the type of data to be accessed from the Metabolomics Workbench, such as metadata or results related to the submitted studies, data from metabolites, genes/proteins and analytical chemistry databases as well as other services related to mass spectrometry and metabolite identification:

context = study | compound | refmet | gene | protein | moverz | exactmass

Part 2: The input specification consists of two required parameters describing the REST request:

input_specification = input_item/input_value

Part 3: The output specification consists of two parameters describing the output generated by the REST request:

output_specification = output_item/(output_format)

The first parameter is required in most cases. The second parameter is optional. The input and output specifications are context sensitive. The context determines the values allowed for the remaining parameters in the input and output specifications as detailed in the sections below.


Setup MW REST base URL...

In [ ]:
MWBaseURL = "https://www.metabolomicsworkbench.org/rest"

**The “moverz” context**

The "moverz" context refers to performing a m/z search against the LIPIDS (a database of ~30,000 computationally generated “bulk” lipid species), MB (the Metabolomics Workbench database of ~64,000 exact structures), or REFMET (a database of ~30,000 standardized names which includes both exact structures and bulk species detected by MS or NMR) databases by specifying an appropriate m/z value, ion type(adduct) and mass tolerance range.

context = moverz

input_item = LIPIDS | MB | REFMET

input_value1 = m/z_value

input_value2 = ion_type_value

input_value3 = m/z_tolerance_value

output_format = txt

The following ion types (adducts) are currently supported: M+H, M+H-H2O, M+2H, M+3H, M+4H, M+K, M+2K, M+Na, M+2Na, M+Li, M+2Li, M+NH4, M+H+CH3CN, M+Na+CH3CN, M.NaFormate+H, M.NH4Formate+H, M.CH3, M.TMSi, M.tBuDMSi, M-H, M-H-H2O, M+Na-2H, M+K-2H, M-2H, M-3H, M-4H, M.Cl, M.F, M.HF2, M.OAc, M.Formate, M.NaFormate-H, M.NH4Formate-H, Neutral.



**Retrieve and process data from a  m/z search in text format**


Setup REST URL to perform MS precursor ion search on Metabolomics Workbench database with m/z 635.52, ion-type M+H and mass tolerance of 0.5 ...

In [ ]:
MWDataURL = MWBaseURL + "/moverz/MB/635.52/M+H/0.5/txt"

Execute REST request using "request" module...

In [ ]:
print("Initiating request: %s" % MWDataURL)
    
Response = requests.get(MWDataURL)

Check "request" status...

In [ ]:
print("\nStatus Code: %d" % (Response.status_code))

if Response.status_code != 200:
    print("Request failed: status_code: %d" % Response.status_code)

Proess search results...

In [ ]:
print("\nAvailable data from a m/z search:\n")

Results = Response.text

LineCount = 0
FilteredResultsLines = []
for Result in Results.split("\n"):
    if len(Result) == 0:
        continue
    if re.search("pre>", Result, re.I):
        continue
    
    LineCount +=1 
    FilteredResultsLines.append(Result)
    
    print("%s" % Result)
    
print("\nTotal number of matches: %d" % (LineCount - 1))


**Load text data into pandas data frame...**


In [ ]:
FILTEREDRESULTSDATA = StringIO("\n".join(FilteredResultsLines))

DataFrame = pd.read_csv(FILTEREDRESULTSDATA, sep="\t", index_col = False)

DataFrame